# Diagnóstico e monitoração da via férrea através de vagão instrumentado

Visando identificar variações dos dados de acelerômetro, para pontuar trechos da via em que existe alguma espécie de anomalia e informar à equipe encarregada pela manutenção da via férrea.

# Importação de Bibliotecas

In [4]:
from copy import deepcopy
import os
import pickle
import sys
from tqdm import tqdm
import numpy as np
import scipy.io as io
import pandas as pd
import h5py
import pyarrow

# Diretório

In [39]:
# Diretório do Pc do Jordan
path = 'H:\\.shortcut-targets-by-id\\1oKn7IN7zznQuhwjDCDdjq8r9wHJYBEhj\\DR_Train\\LRV4313'

#Obtendo o diretório específico para...
acc_path = f'{path}\\accelerometer_data\\' #Acelerômetro
gps_path = f'{path}\\gps_data\\'           #GPS

#Listando o nome dos arquivos das pastas
acc_names = os.listdir(acc_path)
gps_names = os.listdir(gps_path)

# Dados de acelerômetros

Carregando dados dos acelerômetros

In [27]:
print('Carregando os dados dos acelerômetros...')
accel_data = {}
for file in tqdm(acc_names):
    # Separação do nome do arquivo com ('_') como separador: EX. ['3157', '20151022', '3', '2', '5', '2', '5.mat']
    splited_name = file.split('_')
    # Data: Segunda variável
    date = splited_name[1]
    # Nº da passagem: Quarta variável
    daily_passing = splited_name[3]
    # Região: Quinta variável
    region_number = splited_name[4]
    # Direção da viagem: Sexta variável
    running_direction = splited_name[5]
    # Sensor: Sétima variável
    sensor = splited_name[-1][0]

    # Será armazenado somente os dados da região 5:
    if region_number == '5':
        try:    
            with h5py.File(f'{acc_path}/{file}', 'r') as f:
                sensor_data = f.get('save_var_accel')
                sensor_data = np.array(sensor_data)  
        except:
            sensor_data = io.loadmat(f'{acc_path}/{file}')
            sensor_data = sensor_data['save_var_accel'] 

        sensor_data = sensor_data.reshape((-1,))
        sensor_data = sensor_data.astype('float32')

        if not accel_data.get(f'{date}_{daily_passing}_{running_direction}'):
                accel_data[f'{date}_{daily_passing}_{running_direction}'] = {}
                
        accel_data[f'{date}_{daily_passing}_{running_direction}'][f'sensor_{sensor}'] = sensor_data


Concatenação dos dados dos acelerômetros

In [22]:
print('Concatenação dos dados...')

#Concatenação dos dados para montagem do DataFrame
full_df = pd.DataFrame()
for key, values in tqdm(data.items()):
    info = key.split('_')
    partial_df = pd.DataFrame()
    for k in values.keys():
        partial_df[k] = values[k]

    partial_df['date'] = int(info[0])
    partial_df['daily_passing'] = int(info[1])
    partial_df['running_direction'] = int(info[2])
    
    if full_df.empty:
        full_df = partial_df
    else:
        full_df = pd.concat([full_df, partial_df])

full_df.reset_index(inplace=True, drop=True)

In [29]:
nao_nulos = full_df.isnull().sum()
nulos = nao_nulos.sum()
print(f'Número de dados nulos: {nulos}')

# Gravação dos dados de acelerômetro em .parquet
print('Salvando os dados em .parquet...')

# Consultando se existe a pasta no diretório raiz
os.makedirs('accel_data/', exist_ok=True)

prefix = path.split('/')[-1] #Prefixo do nome a ser salvo no arquivo. EX. "LRV4306_acc.parquet"
full_df.to_parquet(f'accel_data/{prefix}_acc.parquet', index=False)

# Dados de GPS

Carregamento dos dados dos GPS

In [35]:
data_gps = {}
print('Carregando os dados dos GPS...')
for file in tqdm(gps_names):
    if file[-4:] == '.ini':
        pass
    else:
        # Separação do nome do arquivo com ('_') como separador: EX.['2769', '20150924', '6', '1', '6', '2gps.mat']
        splited_name = file.split('_') 
        # Data: Segunda variável
        date = splited_name[1] 
        # Nº da passagem: Quarta variável
        daily_passing = splited_name[3]
        # Região: Quinta variável
        region_number = splited_name[4]
        # Direção da viagem: Sétima variável
        running_direction = splited_name[-1][0]
        
        # Será armazenado somente os dados da região 5:
        if region_number == '5':
            gps_data = io.loadmat(f'{gps_path}/{file}')
            gps_data = gps_data['save_var_gps']

            if not data_gps.get(f'{date}_{daily_passing}_{running_direction}'):
                data_gps[f'{date}_{daily_passing}_{running_direction}'] = {}

            for i in range(gps_data.shape[1]):
                data_gps[f'{date}_{daily_passing}_{running_direction}'][f'gps_{i}'] = gps_data[:, i]

Concatenação dos dados de gps

In [37]:
print('Concatenação dos dados...')
#Concatenação dos dados para montagem do DataFrame
full_df = pd.DataFrame()
for key, values in tqdm(data_gps.items()):
    info = key.split('_')
    partial_df = pd.DataFrame()
    for k in values.keys():
        partial_df[k] = values[k]

    partial_df['date'] = int(info[0])
    partial_df['daily_passing'] = int(info[1])
    partial_df['running_direction'] = int(info[2])
    
    if full_df.empty:
        full_df = partial_df
    else:
        full_df = pd.concat([full_df, partial_df])

full_df.reset_index(inplace=True, drop=True)

In [38]:
non_nulos = full_df.isnull().sum()
nulos = non_nulos.sum()
print(f'Número de dados nulos: {nulos}')

# Gravação dos dados de GPS em .parquet
print('Salvando os dados em .parquet...')

# Consultando se existe a pasta no diretório raiz
os.makedirs('gps_data/', exist_ok=True)

prefix = path.split('\\')[-1] #Prefixo do nome a ser salvo no arquivo. EX. "LRV4306_gps.parquet"
full_df.to_parquet(f'gps_data/{prefix}_gps.parquet', index=False)